# Weather Preprocessing

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("juniper")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

args = Array(juniper)
spark = org.apache.spark.sql.SparkSession@7c074a42
session = org.apache.spark.sql.SparkSession@7c074a42
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

## 1. Chargement Weather Raw DataSet

In [3]:
val rawWeatherPath = s"${configuration.common.output.basePath}/common/data/raw_weather.parquet"
val rawWeatherDF = spark.read.parquet(rawWeatherPath)

rawWeatherPath = /home/jovyan/work/output/common/data/raw_weather.parquet
rawWeatherDF = [WBAN: string, Date: string ... 42 more fields]


[WBAN: string, Date: string ... 42 more fields]

In [4]:
rawWeatherDF.count()

4192912

In [5]:
rawWeatherDF.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string

In [6]:
val weatherColumns = Seq(
  "WBAN",
  "Date",
  "Time",
  "StationType",
  "SkyCondition",
  "Visibility",

  /**"most_critical_sky",
  "lowest_cloud_height",
  "ceiling",
  "num_cloud_layers",
  "cloud_risk_score",
  "has_overcast",
  "has_broken",
  "has_obscured",
  "is_clear",
  "has_low_ceiling",
  "visibility_miles",
  "visibility_km",
  "visibility_category",
  "visibility_risk_score",
  "is_low_visibility",
  "is_very_low_visibility",
  "visibility_normalized",
  "visibility_inverse",
  "weather_severity_index",
  "is_vfr_conditions",
  "is_ifr_conditions",
  "requires_cat_ii",
  "operations_risk_level"**/
)

val weatherData = rawWeatherDF
  .select(weatherColumns.head, weatherColumns.tail: _*)
weatherData.show(10)


+-----+--------+----+-----------+------------+----------+
| WBAN|    Date|Time|StationType|SkyCondition|Visibility|
+-----+--------+----+-----------+------------+----------+
|03011|20120101|0015|          0|         CLR|      10.0|
|03011|20120101|0035|          0|         CLR|      10.0|
|03011|20120101|0055|          0|         CLR|      10.0|
|03011|20120101|0115|          0|         CLR|      10.0|
|03011|20120101|0135|          0|         CLR|      10.0|
|03011|20120101|0155|          0|         CLR|      10.0|
|03011|20120101|0215|          0|         CLR|      10.0|
|03011|20120101|0235|          0|         CLR|      10.0|
|03011|20120101|0255|          0|         CLR|      10.0|
|03011|20120101|0315|          0|         CLR|      10.0|
+-----+--------+----+-----------+------------+----------+
only showing top 10 rows



weatherColumns = List(WBAN, Date, Time, StationType, SkyCondition, Visibility)
weatherData = [WBAN: string, Date: string ... 4 more fields]


[WBAN: string, Date: string ... 4 more fields]

# 2. Preprocessing

In [7]:
import com.flightdelay.data.preprocessing.weather.WeatherPreprocessingPipeline

val preprocessedWeatherData = WeatherPreprocessingPipeline.execute()


[Preprocessing] Weather Data Preprocessing Pipeline - Start

Loading raw data from parquet:
  - Path: /home/jovyan/work/output/common/data/raw_weather.parquet
  - Loaded 4192912 raw records

[STEP 2][DataCleaner] Weather Data Cleaning - Start

Original dataset: 4192912 records

Phase 1: Basic Cleaning
  - Current count: 4192907 records

Phase 2: Weather Time Normalization
  - Filtering records at xx:15 only
  - Normalizing time from xx15 to xx00
  - Records before filtering: 4192907
  - Records after filtering: 489665
  - Filtered out: 3703242 (88%)

Phase 3: Data Type Conversion
Conversion des types de données: WindDirection, DewPointFarenheit, WetBulbCelsius, DryBulbFarenheit, StationType, RelativeHumidity, Altimeter, DryBulbCelsius, WBAN, WindSpeed, Visibility, StationPressure, DewPointCelsius, Time, WetBulbFarenheit
  - Converting Date from YYYYMMDD to Date type
  - Current count: 489665 records

Phase 4: Final Validation
  - Validation passed: 489665 records

Cleaning Summary
Ori

preprocessedWeatherData = [WBAN: string, Date: date ... 65 more fields]


[WBAN: string, Date: date ... 65 more fields]

In [9]:
val weatherColumns = Seq(
  "WBAN",
  "Date",
  "Time",
  "StationType",
  "SkyCondition",
  "Visibility",

  "feature_ceiling",  
  /**"feature_most_critical_sky",
  "feature_lowest_cloud_height",
  
  "feature_num_cloud_layers",
  "feature_cloud_risk_score",
  "feature_has_overcast",
  "feature_has_broken",
  "feature_has_obscured",
  "feature_is_clear",
  "feature_has_low_ceiling",
  "feature_visibility_miles",
  "feature_visibility_km",
  "feature_visibility_category",
  "feature_visibility_risk_score",
  "feature_is_low_visibility",
  "feature_is_very_low_visibility",
  "feature_visibility_normalized",
  "feature_visibility_inverse",
  "feature_weather_severity_index",
  "feature_is_vfr_conditions",
  "feature_is_ifr_conditions",
  "feature_requires_cat_ii",
  "feature_operations_risk_level"**/
)

val weatherData = preprocessedWeatherData
  .select(weatherColumns.head, weatherColumns.tail: _*)
  .orderBy("WBAN", "Date", "Time")
weatherData.show(10)


+-----+----------+----+-----------+------------+----------+---------------+
| WBAN|      Date|Time|StationType|SkyCondition|Visibility|feature_ceiling|
+-----+----------+----+-----------+------------+----------+---------------+
|03011|2012-01-01|0000|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0100|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0200|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0300|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0400|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0500|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0600|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0700|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0800|          0|         CLR|      10.0|          99999|
|03011|2012-01-01|0900|          0|         CLR|      10.0|          99999|
+-----+-----

weatherColumns = List(WBAN, Date, Time, StationType, SkyCondition, Visibility, feature_ceiling)
weatherData = [WBAN: string, Date: date ... 5 more fields]


[WBAN: string, Date: date ... 5 more fields]